In [ ]:
%%capture


In [2]:
import yfinance as yf
import pandas as pd
from transformers import pipeline


In [3]:
def get_ticker_news_sentiment(ticker):
    """
    Returns a Pandas dataframe of the given ticker's most recent news article headlines,
    with the overal sentiment of each article.

    Args:
        ticker (string)

    Returns:
        pd.DataFrame: {'Date', 'Article title', Article sentiment'}
    """
    ticker_news = yf.Ticker(ticker)
    news_list = ticker_news.get_news()
    pipe = pipeline("text-classification", model="ProsusAI/finbert")

    data = []
    for article in news_list:
        publish_data = article['content']['pubDate']
        article_summary = article['content']['summary']
        data.append({'Date':f'{publish_data}',
                         'Article summary':f'{article_summary}',
                         'sentiment_score':f'{pipe(article_summary)[0]["score"]}',
                         'Sentiment_label':f'{pipe(article_summary)[0]["label"]}'})
    df = pd.DataFrame(data)
    return df


In [42]:
df = get_ticker_news_sentiment('AAPL')
df

,Date,Article summary,sentiment_score,Sentiment_label
0,2025-03-09T14:00:00Z,"The world is bad sometimes, but it feels even ...",0.608728289604187,neutral
1,2025-03-09T20:12:21Z,We recently published a list of Top 10 Stocks ...,0.8741039037704468,negative
2,2025-03-09T18:01:22Z,Hopes that President Trump could take a softer...,0.9387659430503845,negative
3,2025-03-09T14:07:00Z,One of tech's old guard has made it clear that...,0.7316726446151733,neutral
4,2025-03-09T10:50:00Z,Artificial intelligence is already improving t...,0.8931741714477539,positive
5,2025-03-08T20:57:06Z,We recently published a list of Top 10 AI Stoc...,0.9170315265655518,neutral
6,2025-03-08T16:51:39Z,“Now Streaming” is The Fly’s weekly recap of t...,0.9030751585960388,neutral
7,2025-03-08T15:43:24Z,We recently compiled a list of the 10 High-Pot...,0.7345390915870667,positive
8,2025-03-08T14:30:00Z,Warren Buffett is responsible for putting up t...,0.8026019930839539,neutral
9,2025-03-08T14:01:36Z,Champions and critics of DEI policies are agai...,0.5408843755722046,negative


In [43]:
sentiment_weights = {
    "positive": 1.0,
    "neutral": 0.0,
    "negative": -1.0
}

In [44]:
df['sentiment_weight'] = df['Sentiment_label'].map(sentiment_weights)

In [45]:
df

,Date,Article summary,sentiment_score,Sentiment_label,sentiment_weight
0,2025-03-09T14:00:00Z,"The world is bad sometimes, but it feels even ...",0.608728289604187,neutral,0.0
1,2025-03-09T20:12:21Z,We recently published a list of Top 10 Stocks ...,0.8741039037704468,negative,-1.0
2,2025-03-09T18:01:22Z,Hopes that President Trump could take a softer...,0.9387659430503845,negative,-1.0
3,2025-03-09T14:07:00Z,One of tech's old guard has made it clear that...,0.7316726446151733,neutral,0.0
4,2025-03-09T10:50:00Z,Artificial intelligence is already improving t...,0.8931741714477539,positive,1.0
5,2025-03-08T20:57:06Z,We recently published a list of Top 10 AI Stoc...,0.9170315265655518,neutral,0.0
6,2025-03-08T16:51:39Z,“Now Streaming” is The Fly’s weekly recap of t...,0.9030751585960388,neutral,0.0
7,2025-03-08T15:43:24Z,We recently compiled a list of the 10 High-Pot...,0.7345390915870667,positive,1.0
8,2025-03-08T14:30:00Z,Warren Buffett is responsible for putting up t...,0.8026019930839539,neutral,0.0
9,2025-03-08T14:01:36Z,Champions and critics of DEI policies are agai...,0.5408843755722046,negative,-1.0


In [55]:
df['sentiment_score'] = pd.to_numeric(df['sentiment_score'], errors='coerce')

In [56]:
result = (df['sentiment_score'] * df['sentiment_weight']).sum()

In [57]:
result

-0.7260409593582153

In [68]:
ticker_list = {'AMPY':0.0, 'BOOM':0.0, 'BWB':0.0}


In [70]:
for ticker in ticker_list.keys():
    df = get_ticker_news_sentiment(ticker)
    df['sentiment_weight'] = df['Sentiment_label'].map(sentiment_weights)
    df['sentiment_score'] = pd.to_numeric(df['sentiment_score'], errors='coerce')
    result = (df['sentiment_score'] * df['sentiment_weight']).sum()
    ticker_list[ticker] = result

In [71]:
ticker_list

{'AMPY': -1.3335750699043274,
 'BOOM': 0.36433637142181396,
 'BWB': 1.3139823079109192}

In [72]:
recommendation_stock = dict(sorted(ticker_list.items(), key=lambda item: item[1], reverse=True))

In [73]:
recommendation_stock

{'BWB': 1.3139823079109192,
 'BOOM': 0.36433637142181396,
 'AMPY': -1.3335750699043274}

In [ ]:
ticker = 'AAPL'
df.to_csv(f'{ticker}_sentiment.csv', index=False)

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('../src/database/valid_tickers.csv')

In [9]:
valid_stocks = df['ticker'].tolist()

In [11]:
len(valid_stocks)

31626

In [12]:
val = {"stocks": valid_stocks}

In [13]:
val

{'stocks': ['AAPL',
  'T',
  'GOOG',
  'MO',
  'AA',
  'AXP',
  'BABA',
  'ABT',
  'AMAT',
  'AMGN',
  'AIG',
  'ALL',
  'ADBE',
  'GOOGL',
  'ACN',
  'ABBV',
  'MT',
  'LLY',
  'APA',
  'ADP',
  'AKAM',
  'NLY',
  'ATVI',
  'ADSK',
  'ADM',
  'WBA',
  'PANW',
  'AMD',
  'AVGO',
  'EA',
  'AEM',
  'APD',
  'AMBA',
  'NVS',
  'LULU',
  'BRK.AX',
  'ARRY',
  'A',
  'ORLY',
  'AZO',
  'AN',
  'AZN',
  'BUD',
  'BDX',
  'AB',
  'CS',
  'AFL',
  'ADI',
  'ACIW',
  'AMP',
  'AMTD',
  'AEO',
  'NVO',
  'ALTR',
  'PAA',
  'AAP',
  'FNMA',
  'UBS',
  'ARLP',
  'ABC',
  'ATI',
  'ADT',
  'AVB',
  'LH',
  'AVY',
  'ABMD',
  'AON',
  'ADC',
  'AYI',
  'ASML',
  'AMT',
  'ACM',
  'ARI',
  'AR',
  'AAN',
  'BAH',
  'AXAS',
  'ALB',
  'AIZ',
  'SAIC',
  'CAR',
  'AU',
  'APH',
  'AMX',
  'JKHY',
  'AMKR',
  'AEIS',
  'VRSK',
  'APO',
  'RBA',
  'MAA',
  'ASX',
  'ARCO',
  'ANET',
  'AIR',
  'WAB',
  'RS',
  'PKG',
  'AMG',
  'ALSN',
  'ALGN',
  'ROK',
  'GLPI',
  'AZZ',
  'ARMK',
  'AOS',
  'AMCX',
 

In [14]:
import json

In [ ]:
with open('../src/database/valid_stocks.json', 'w') as f:
    json.dump(val, f)

In [20]:
path = "../src/data/valid_stocks.json"

In [21]:
with open(path, 'r') as f:
    symbols = json.load(f)


In [28]:
x = set(symbols['stocks'])

In [29]:
import re

In [10]:
def is_empty(input):
        """Check if the 'symbols' field is empty."""
        if not input["symbols"]:  # Check if the list is empty or None
            return "ResponseEnum.EMPTY_LIST.value"
        return None  # No error if symbols are not empty


In [7]:
def is_valid_symbols(symbol):
        """Check if the symbol is valid (i.e., present in the loaded symbols)."""
        if not symbol:  # Additional check for empty symbol
            return "ResponseEnum.EMPTY_SYMBOL.value"
        if symbol not in ["s", "y","x"]:
            return "ResponseEnum.INVALID_SYMBOLS.value.format(symbol)"
        return "ok"  # No error if the symbol is valid


In [12]:
X = { "symbols": [] }


In [13]:
print(is_empty(X))

ResponseEnum.EMPTY_LIST.value


In [ ]:
if None:
     True